# Fourier Flows (Casseau Ch. 3.2.1)

This notebook reproduces the first two Fourier-flow verification cases from Casseau (pure N2 and homogeneous air).
We use the current diffusion model and the new Maxwell-slip / Smoluchowski-jump wall BC.

Notes:
- Case 1: pure N2 (Knov = 0.1).
- Case 2a: homogeneous air (0.5 N2 / 0.5 O2), Knov = 0.002.
- Case 2b: homogeneous air, Knov = 0.1.

You can paste digitized Casseau datapoints into the reference arrays below to overlay in the plots.


In [1]:
%load_ext autoreload
%autoreload 2

from __future__ import annotations

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import fourier_flows_helpers as ff_helpers

import plotly.io as pio
pio.templates.default = "plotly_white"


## Case 1: Pure N2 (Kn = 0.1)


In [69]:
from compressible_core import constants

T = 3000.0
d = 4.17e-10  # m, hard-sphere diameter for N2 from Casseau Tab A1
n = 2.086e19
m = 28.0 / 1000 / constants.N_A  # kg, mass of N2 molecule
dx = 1.0 / 21

lambda_hs = 1.0 / (np.sqrt(2) * np.pi * d**2 * n)
c_bar = np.sqrt(8.0 * constants.k * T / (np.pi * m))
tau_collision = lambda_hs / c_bar
tau_residence = dx / c_bar

print(f"lambda_hs = {lambda_hs:.3e} m")
print(f"c_bar = {c_bar:.3e} m/s")
print(f"tau_residence = {tau_residence:.3e} s")
print(f"tau_collision = {tau_collision:.3e} s")

lambda_hs = 6.205e-02 m
c_bar = 1.506e+03 m/s
tau_residence = 3.162e-05 s
tau_collision = 4.120e-05 s


In [9]:
# Case 1 settings (number densities in 1/m^3)
case1_enabled = True
case1_species_names = ("N2",)
case1_n_by_species = [2.086e19]
case1_run_kwargs = {
    "nx": 1,
    "ny": 101,  # set higher for smoother discretization
    "Lx": 1.0,
    "H": 1.0,
    "U": 300.0,
    "Tw_bottom": 2000.0,
    "Tw_top": 3000.0,
    "T_init": 2500.0,
    "t_final": 5e-3,
    "save_interval": 1,
    "dt": 1e-6,  # set higher for smoother discretization
}

case1 = None
if case1_enabled:
    case1 = ff_helpers.run_case(
        case1_species_names,
        case1_n_by_species,
        "case1",
        **case1_run_kwargs,
    )


In [22]:
PROFILE_SETTINGS = {
    "x_target": None,
}

PLOT_SETTINGS = {
    "figsize": (10, 4),
    "sharey": True,
}

# cases = [c for c in (case1, case2a, case2b) if c is not None]
cases = [case1]

N_TIME_SAMPLES = 10

repo_root = ff_helpers.find_repo_root()
REF_TR_TB_PATH = "/home/hhoechter/tum/jaxfluids_internship/experiments/fourier_flows/casseau_figure_3_8_case_1_tr_tb.csv"
REF_TV_TB_PATH = "/home/hhoechter/tum/jaxfluids_internship/experiments/fourier_flows/casseau_figure_3_8_case_1_tv_tb.csv"


def load_reference_csv(path):
    path = str(path)
    try:
        with open(path, "r") as f:
            header = f.readline().strip()
    except FileNotFoundError:
        return []

    dataset_names = [name for name in header.split(",") if name]
    data = np.genfromtxt(path, delimiter=",", skip_header=2)
    data = np.atleast_2d(data)

    refs = []
    n_pairs = data.shape[1] // 2
    for i in range(n_pairs):
        name = dataset_names[i] if i < len(dataset_names) else f"dataset_{i+1}"
        x = data[:, 2 * i]
        y = data[:, 2 * i + 1]
        mask = np.isfinite(x) & np.isfinite(y)
        if not np.any(mask):
            continue
        refs.append({"name": name, "x": x[mask], "y": y[mask]})
    return refs


def parse_ref_name(name: str):
    lower = name.lower()
    if "dsmc" in lower:
        method = "dsmc"
    elif "hy2foam" in lower:
        method = "hy2foam"
    else:
        method = name

    species = None
    if "_n2" in lower:
        species = "N2"
    elif "_o2" in lower:
        species = "O2"

    return method, species


ref_tr = load_reference_csv(REF_TR_TB_PATH)
ref_tv = load_reference_csv(REF_TV_TB_PATH)

profiles_by_time = {}
for case in cases:
    if case is None:
        continue
    U_hist = case["U_hist"]
    t_hist = np.asarray(case["t_hist"])
    n_steps = len(U_hist)
    if n_steps == 0:
        continue
    sample_count = min(N_TIME_SAMPLES, n_steps)
    indices = np.linspace(0, n_steps - 1, sample_count, dtype=int)
    indices = np.unique(indices)

    samples = []
    for idx in indices:
        y, Tn, Tv = ff_helpers.extract_profile_from_U(
            case, U_hist[idx], x_target=PROFILE_SETTINGS["x_target"]
        )
        samples.append((idx, float(t_hist[idx]), y, Tn, Tv))
    profiles_by_time[case["name"]] = samples

case_color_cycle = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"]
case_colors = {
    name: case_color_cycle[i % len(case_color_cycle)]
    for i, name in enumerate(profiles_by_time.keys())
}
ref_colors = {
    "dsmc": "#111111",
    "hy2foam": "#7a3db8",
}
ref_symbols = {
    "N2": "circle-open",
    "O2": "triangle-up-open",
    None: "circle",
}

fig = make_subplots(
    rows=1,
    cols=2,
    shared_yaxes=PLOT_SETTINGS["sharey"],
    subplot_titles=("Normalized T_tr", "Normalized T_v"),
)

# T_tr profiles (multiple time steps)
for name, samples in profiles_by_time.items():
    final_idx = samples[-1][0]
    color = case_colors.get(name, "#1f77b4")
    for idx, t, y, Tn, _Tv in samples:
        is_final = idx == final_idx
        fig.add_trace(
            go.Scatter(
                x=Tn,
                y=y,
                mode="lines",
                name=f"{name} t={t:.2e}s" if not is_final else f"{name} (final)",
                legendgroup=name,
                showlegend=bool(is_final),
                opacity=1.0 if is_final else 0.35,
                line=dict(width=2 if is_final else 1, color=color),
                customdata=np.full_like(y, t, dtype=float),
            ),
            row=1,
            col=1,
        )

shown_ref = set()
for ref in ref_tr:
    method, species = parse_ref_name(ref["name"])
    color = ref_colors.get(method, "#444444")
    show = method not in shown_ref
    shown_ref.add(method)
    fig.add_trace(
        go.Scatter(
            x=ref["x"],
            y=ref["y"],
            mode="markers",
            name=f"Casseau {method}",
            legendgroup=f"casseau-{method}",
            showlegend=show,
            marker=dict(size=6, color=color, symbol=ref_symbols.get(species)),
            text=[species or ""] * len(ref["x"]),
        ),
        row=1,
        col=1,
    )

# T_v profiles (multiple time steps)
for name, samples in profiles_by_time.items():
    final_idx = samples[-1][0]
    color = case_colors.get(name, "#1f77b4")
    for idx, t, y, _Tn, Tv in samples:
        is_final = idx == final_idx
        fig.add_trace(
            go.Scatter(
                x=Tv,
                y=y,
                mode="lines",
                name=f"{name} t={t:.2e}s" if not is_final else f"{name} (final)",
                legendgroup=name,
                showlegend=False,
                opacity=1.0 if is_final else 0.35,
                line=dict(width=2 if is_final else 1, color=color),
                customdata=np.full_like(y, t, dtype=float),
            ),
            row=1,
            col=2,
        )

for ref in ref_tv:
    method, species = parse_ref_name(ref["name"])
    color = ref_colors.get(method, "#444444")
    show = method not in shown_ref
    shown_ref.add(method)
    fig.add_trace(
        go.Scatter(
            x=ref["x"],
            y=ref["y"],
            mode="markers",
            name=f"Casseau {method}",
            legendgroup=f"casseau-{method}",
            showlegend=show,
            marker=dict(size=6, color=color, symbol=ref_symbols.get(species)),
            text=[species or ""] * len(ref["x"]),
        ),
        row=1,
        col=2,
    )

fig.update_xaxes(title_text="T_tr / T_b0", row=1, col=1, range=[0.98, 1.52])
fig.update_xaxes(title_text="T_v / T_b0", row=1, col=2, range=[0.98, 1.52])
fig.update_yaxes(title_text="y / H", row=1, col=1)

fig.update_layout(
    width=1000,
    height=500,
    legend_title_text="Cases",
    legend=dict(
        x=0.5,
        y=-0.2,
        xanchor="center",
        yanchor="top",
        orientation="h",
    ),
)

fig.show()


## Case 2a: Homogeneous Air (Kn = 0.002)


In [28]:
from compressible_core import constants

T = 3000.0
d = 4.17e-10  # m, hard-sphere diameter for N2 from Casseau Tab A1, d_O2 = 4.07e-10

n = 2* 52.15e19
m = (28.0+31.0)/2  / 1000 / constants.N_A  # kg, mass of N2 molecule
dx = 1.0 / 201

lambda_hs = 1.0 / (np.sqrt(2) * np.pi * d**2 * n)
c_bar = np.sqrt(8.0 * constants.k * T / (np.pi * m))
tau_collision = lambda_hs / c_bar
tau_residence = dx / c_bar

print(f"lambda_hs = {lambda_hs:.3e} m")
print(f"c_bar = {c_bar:.3e} m/s")
print(f"tau_residence = {tau_residence:.3e} s")
print(f"tau_collision = {tau_collision:.3e} s")

# conclusion: 
# dx must be a fraction of lambda -> dx = 5e-4 m -> ny = 2001
# dt must be a fraction of tau_collision -> dt = 1e-7 s -> t_final = 1e-4 s

lambda_hs = 1.241e-03 m
c_bar = 1.467e+03 m/s
tau_residence = 3.391e-06 s
tau_collision = 8.457e-07 s


In [2]:
# Case 2a settings (number densities in 1/m^3)
case2a_species_names = ("N2", "O2")
case2a_n_by_species = [52.15e19, 52.15e19]
case2a_run_kwargs = {
    "nx": 1,
    "ny": 21,
    "Lx": 1.0,
    "H": 1.0,
    "U": 300.0,
    "Tw_bottom": 2000.0,
    "Tw_top": 3000.0,
    "T_init": 2500.0,
    "t_final": 1e-2,
    "save_interval": 10,
    "dt": 1e-5,
}

case2a = ff_helpers.run_case(
    case2a_species_names,
    case2a_n_by_species,
    "case2a",
    **case2a_run_kwargs,
)


In [35]:
PROFILE_SETTINGS = {
    "x_target": None,
}

PLOT_SETTINGS = {
    "figsize": (10, 4),
    "sharey": True,
}

# cases = [c for c in (case1, case2a, case2b) if c is not None]
cases = [case2a]

N_TIME_SAMPLES = 10

repo_root = ff_helpers.find_repo_root()
REF_TR_TB_PATH = "/home/hhoechter/tum/jaxfluids_internship/experiments/fourier_flows/casseau_figure_3_8_case_2a_tr_tb.csv"
REF_TV_TB_PATH = "/home/hhoechter/tum/jaxfluids_internship/experiments/fourier_flows/casseau_figure_3_8_case_2a_tv_tb.csv"


def load_reference_csv(path):
    path = str(path)
    try:
        with open(path, "r") as f:
            header = f.readline().strip()
    except FileNotFoundError:
        return []

    dataset_names = [name for name in header.split(",") if name]
    data = np.genfromtxt(path, delimiter=",", skip_header=2)
    data = np.atleast_2d(data)

    refs = []
    n_pairs = data.shape[1] // 2
    for i in range(n_pairs):
        name = dataset_names[i] if i < len(dataset_names) else f"dataset_{i+1}"
        x = data[:, 2 * i]
        y = data[:, 2 * i + 1]
        mask = np.isfinite(x) & np.isfinite(y)
        if not np.any(mask):
            continue
        refs.append({"name": name, "x": x[mask], "y": y[mask]})
    return refs


def parse_ref_name(name: str):
    lower = name.lower()
    if "dsmc" in lower:
        method = "dsmc"
    elif "hy2foam" in lower:
        method = "hy2foam"
    else:
        method = name

    species = None
    if "_n2" in lower:
        species = "N2"
    elif "_o2" in lower:
        species = "O2"

    return method, species


ref_tr = load_reference_csv(REF_TR_TB_PATH)
ref_tv = load_reference_csv(REF_TV_TB_PATH)

profiles_by_time = {}
for case in cases:
    if case is None:
        continue
    U_hist = case["U_hist"]
    t_hist = np.asarray(case["t_hist"])
    n_steps = len(U_hist)
    if n_steps == 0:
        continue
    sample_count = min(N_TIME_SAMPLES, n_steps)
    indices = np.linspace(0, n_steps - 1, sample_count, dtype=int)
    indices = np.unique(indices)

    samples = []
    for idx in indices:
        y, Tn, Tv = ff_helpers.extract_profile_from_U(
            case, U_hist[idx], x_target=PROFILE_SETTINGS["x_target"]
        )
        samples.append((idx, float(t_hist[idx]), y, Tn, Tv))
    profiles_by_time[case["name"]] = samples

case_color_cycle = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"]
case_colors = {
    name: case_color_cycle[i % len(case_color_cycle)]
    for i, name in enumerate(profiles_by_time.keys())
}
ref_colors = {
    "dsmc": "#111111",
    "hy2foam": "#7a3db8",
}
ref_symbols = {
    "N2": "circle-open",
    "O2": "triangle-up-open",
    None: "circle",
}

fig = make_subplots(
    rows=1,
    cols=2,
    shared_yaxes=PLOT_SETTINGS["sharey"],
    subplot_titles=("Normalized T_tr", "Normalized T_v"),
)

# T_tr profiles (multiple time steps)
for name, samples in profiles_by_time.items():
    final_idx = samples[-1][0]
    color = case_colors.get(name, "#1f77b4")
    for idx, t, y, Tn, _Tv in samples:
        is_final = idx == final_idx
        fig.add_trace(
            go.Scatter(
                x=Tn,
                y=y,
                mode="lines",
                name=f"{name} t={t:.2e}s" if not is_final else f"{name} (final)",
                legendgroup=name,
                showlegend=bool(is_final),
                opacity=1.0 if is_final else 0.35,
                line=dict(width=2 if is_final else 1, color=color),
                customdata=np.full_like(y, t, dtype=float),
            ),
            row=1,
            col=1,
        )

shown_ref = set()
for ref in ref_tr:
    method, species = parse_ref_name(ref["name"])
    color = ref_colors.get(method, "#444444")
    show = method not in shown_ref
    shown_ref.add(method)
    fig.add_trace(
        go.Scatter(
            x=ref["x"],
            y=ref["y"],
            mode="markers",
            name=f"Casseau {method}",
            legendgroup=f"casseau-{method}",
            showlegend=show,
            marker=dict(size=6, color=color, symbol=ref_symbols.get(species)),
            text=[species or ""] * len(ref["x"]),
        ),
        row=1,
        col=1,
    )

# T_v profiles (multiple time steps)
for name, samples in profiles_by_time.items():
    final_idx = samples[-1][0]
    color = case_colors.get(name, "#1f77b4")
    for idx, t, y, _Tn, Tv in samples:
        is_final = idx == final_idx
        fig.add_trace(
            go.Scatter(
                x=Tv,
                y=y,
                mode="lines",
                name=f"{name} t={t:.2e}s" if not is_final else f"{name} (final)",
                legendgroup=name,
                showlegend=False,
                opacity=1.0 if is_final else 0.35,
                line=dict(width=2 if is_final else 1, color=color),
                customdata=np.full_like(y, t, dtype=float),
            ),
            row=1,
            col=2,
        )

for ref in ref_tv:
    method, species = parse_ref_name(ref["name"])
    color = ref_colors.get(method, "#444444")
    show = method not in shown_ref
    shown_ref.add(method)
    fig.add_trace(
        go.Scatter(
            x=ref["x"],
            y=ref["y"],
            mode="markers",
            name=f"Casseau {method}",
            legendgroup=f"casseau-{method}",
            showlegend=show,
            marker=dict(size=6, color=color, symbol=ref_symbols.get(species)),
            text=[species or ""] * len(ref["x"]),
        ),
        row=1,
        col=2,
    )

fig.update_xaxes(title_text="T_tr / T_b0", row=1, col=1, range=[0.98, 1.52])
fig.update_xaxes(title_text="T_v / T_b0", row=1, col=2, range=[0.98, 1.52])
fig.update_yaxes(title_text="y / H", row=1, col=1)

fig.update_layout(
    width=1000,
    height=500,
    legend_title_text="Cases",
    legend=dict(
        x=0.5,
        y=-0.2,
        xanchor="center",
        yanchor="top",
        orientation="h",
    ),
)

fig.show()


In [34]:
# Thermal conductivity profiles for Case 2a
N_TIME_SAMPLES = 10

U_hist = case2a["U_hist"]
t_hist = np.asarray(case2a["t_hist"])
n_steps = len(U_hist)
sample_count = min(N_TIME_SAMPLES, n_steps)
indices = np.linspace(0, n_steps - 1, sample_count, dtype=int)
indices = np.unique(indices)

fig = make_subplots(
    rows=1,
    cols=2,
    shared_yaxes=True,
    subplot_titles=("eta_tr (trans-rot)", "eta_v (vib)"),
)

for idx in indices:
    t = float(t_hist[idx])
    is_final = idx == indices[-1]
    y, eta_tr, eta_v = ff_helpers.extract_thermal_conductivity_profile_from_U(
        case2a, U_hist[idx]
    )
    fig.add_trace(
        go.Scatter(
            x=eta_tr,
            y=y,
            mode="lines",
            name=f"t={t:.2e}s",
            legendgroup="eta_tr",
            showlegend=bool(is_final),
            opacity=1.0 if is_final else 0.35,
            line=dict(width=2 if is_final else 1, color="#1f77b4"),
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=eta_v,
            y=y,
            mode="lines",
            name=f"t={t:.2e}s",
            legendgroup="eta_v",
            showlegend=False,
            opacity=1.0 if is_final else 0.35,
            line=dict(width=2 if is_final else 1, color="#ff7f0e"),
        ),
        row=1,
        col=2,
    )

fig.update_xaxes(title_text="eta_tr [W/(m K)]", row=1, col=1)
fig.update_xaxes(title_text="eta_v [W/(m K)]", row=1, col=2)
fig.update_yaxes(title_text="y / H", row=1, col=1)
fig.update_layout(
    width=1000,
    height=500,
    title="Thermal conductivity – Case 2a",
    legend=dict(x=0.5, y=-0.15, xanchor="center", yanchor="top", orientation="h"),
)
fig.show()


## Case 2b: Homogeneous Air (Kn = 0.1)


In [ ]:
from compressible_core import constants

T = 3000.0
d = 4.17e-10  # m, hard-sphere diameter for N2 from Casseau Tab A1, d_O2 = 4.07e-10

n = 2* 1.043e19
m = (28.0+31.0)/2  / 1000 / constants.N_A  # kg, mass of N2 molecule


lambda_hs = 1.0 / (np.sqrt(2) * np.pi * d**2 * n)
dx = (lambda_hs / 5)

c_bar = np.sqrt(8.0 * constants.k * T / (np.pi * m))
tau_collision = lambda_hs / c_bar
tau_residence = dx / c_bar

print(f"lambda_hs = {lambda_hs:.3e} m")
print(f"dx = {dx:.3e} m")
print(f"c_bar = {c_bar:.3e} m/s")
print(f"tau_residence = {tau_residence:.3e} s")
print(f"tau_collision = {tau_collision:.3e} s")

# conclusion: 
# dx must be a fraction of lambda -> dx = 1e-2 m -> ny = 101
# dt must be a fraction of tau_collision -> dt = 1e-6 s -> t_final = 1e-4 s

lambda_hs = 6.205e-02 m
dx = 1.241e-02 m
c_bar = 1.467e+03 m/s
tau_residence = 8.457e-06 s
tau_collision = 4.229e-05 s


In [87]:
# Case 2b settings (number densities in 1/m^3)
case2b_species_names = ("N2", "O2")
case2b_n_by_species = [1.043e19, 1.043e19]
case2b_run_kwargs = {
    "nx": 1,
    "ny": 101,
    "Lx": 1.0,
    "H": 1.0,
    "U": 300.0,
    "Tw_bottom": 2000.0,
    "Tw_top": 3000.0,
    "T_init": 2500.0,
    "t_final": 1e-4,
    "save_interval": 10,
    "dt": 1e-6,
}

case2b = ff_helpers.run_case(
    case2b_species_names,
    case2b_n_by_species,
    "case2b",
    **case2b_run_kwargs,
)


In [89]:
PROFILE_SETTINGS = {
    "x_target": None,
}

PLOT_SETTINGS = {
    "figsize": (10, 4),
    "sharey": True,
}

# cases = [c for c in (case1, case2a, case2b) if c is not None]
cases = [case2b]

N_TIME_SAMPLES = 10

repo_root = ff_helpers.find_repo_root()
REF_TR_TB_PATH = (
    repo_root / "experiments" / "fourier_flows" / "casseau_figure_3_8_case_2a_tr_tb.csv"
)
REF_TV_TB_PATH = (
    repo_root / "experiments" / "fourier_flows" / "casseau_figure_3_8_case_2a_tv_tb.csv"
)


def load_reference_csv(path):
    path = str(path)
    try:
        with open(path, "r") as f:
            header = f.readline().strip()
    except FileNotFoundError:
        return []

    dataset_names = [name for name in header.split(",") if name]
    data = np.genfromtxt(path, delimiter=",", skip_header=2)
    data = np.atleast_2d(data)

    refs = []
    n_pairs = data.shape[1] // 2
    for i in range(n_pairs):
        name = dataset_names[i] if i < len(dataset_names) else f"dataset_{i+1}"
        x = data[:, 2 * i]
        y = data[:, 2 * i + 1]
        mask = np.isfinite(x) & np.isfinite(y)
        if not np.any(mask):
            continue
        refs.append({"name": name, "x": x[mask], "y": y[mask]})
    return refs


def parse_ref_name(name: str):
    lower = name.lower()
    if "dsmc" in lower:
        method = "dsmc"
    elif "hy2foam" in lower:
        method = "hy2foam"
    else:
        method = name

    species = None
    if "_n2" in lower:
        species = "N2"
    elif "_o2" in lower:
        species = "O2"

    return method, species


ref_tr = load_reference_csv(REF_TR_TB_PATH)
ref_tv = load_reference_csv(REF_TV_TB_PATH)

profiles_by_time = {}
for case in cases:
    if case is None:
        continue
    U_hist = case["U_hist"]
    t_hist = np.asarray(case["t_hist"])
    n_steps = len(U_hist)
    if n_steps == 0:
        continue
    sample_count = min(N_TIME_SAMPLES, n_steps)
    indices = np.linspace(0, n_steps - 1, sample_count, dtype=int)
    indices = np.unique(indices)

    samples = []
    for idx in indices:
        y, Tn, Tv = ff_helpers.extract_profile_from_U(
            case, U_hist[idx], x_target=PROFILE_SETTINGS["x_target"]
        )
        samples.append((idx, float(t_hist[idx]), y, Tn, Tv))
    profiles_by_time[case["name"]] = samples

case_color_cycle = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"]
case_colors = {
    name: case_color_cycle[i % len(case_color_cycle)]
    for i, name in enumerate(profiles_by_time.keys())
}
ref_colors = {
    "dsmc": "#111111",
    "hy2foam": "#7a3db8",
}
ref_symbols = {
    "N2": "circle-open",
    "O2": "triangle-up-open",
    None: "circle",
}

fig = make_subplots(
    rows=1,
    cols=2,
    shared_yaxes=PLOT_SETTINGS["sharey"],
    subplot_titles=("Normalized T_tr", "Normalized T_v"),
)

# T_tr profiles (multiple time steps)
for name, samples in profiles_by_time.items():
    final_idx = samples[-1][0]
    color = case_colors.get(name, "#1f77b4")
    for idx, t, y, Tn, _Tv in samples:
        is_final = idx == final_idx
        fig.add_trace(
            go.Scatter(
                x=Tn,
                y=y,
                mode="lines",
                name=f"{name} t={t:.2e}s" if not is_final else f"{name} (final)",
                legendgroup=name,
                showlegend=bool(is_final),
                opacity=1.0 if is_final else 0.35,
                line=dict(width=2 if is_final else 1, color=color),
                customdata=np.full_like(y, t, dtype=float),
            ),
            row=1,
            col=1,
        )

shown_ref = set()
for ref in ref_tr:
    method, species = parse_ref_name(ref["name"])
    color = ref_colors.get(method, "#444444")
    show = method not in shown_ref
    shown_ref.add(method)
    fig.add_trace(
        go.Scatter(
            x=ref["x"],
            y=ref["y"],
            mode="markers",
            name=f"Casseau {method}",
            legendgroup=f"casseau-{method}",
            showlegend=show,
            marker=dict(size=6, color=color, symbol=ref_symbols.get(species)),
            text=[species or ""] * len(ref["x"]),
        ),
        row=1,
        col=1,
    )

# T_v profiles (multiple time steps)
for name, samples in profiles_by_time.items():
    final_idx = samples[-1][0]
    color = case_colors.get(name, "#1f77b4")
    for idx, t, y, _Tn, Tv in samples:
        is_final = idx == final_idx
        fig.add_trace(
            go.Scatter(
                x=Tv,
                y=y,
                mode="lines",
                name=f"{name} t={t:.2e}s" if not is_final else f"{name} (final)",
                legendgroup=name,
                showlegend=False,
                opacity=1.0 if is_final else 0.35,
                line=dict(width=2 if is_final else 1, color=color),
                customdata=np.full_like(y, t, dtype=float),
            ),
            row=1,
            col=2,
        )

for ref in ref_tv:
    method, species = parse_ref_name(ref["name"])
    color = ref_colors.get(method, "#444444")
    show = method not in shown_ref
    shown_ref.add(method)
    fig.add_trace(
        go.Scatter(
            x=ref["x"],
            y=ref["y"],
            mode="markers",
            name=f"Casseau {method}",
            legendgroup=f"casseau-{method}",
            showlegend=show,
            marker=dict(size=6, color=color, symbol=ref_symbols.get(species)),
            text=[species or ""] * len(ref["x"]),
        ),
        row=1,
        col=2,
    )

fig.update_xaxes(title_text="T_tr / T_b0", row=1, col=1, range=[0.98, 1.52])
fig.update_xaxes(title_text="T_v / T_b0", row=1, col=2, range=[0.98, 1.52])
fig.update_yaxes(title_text="y / H", row=1, col=1)

fig.update_layout(
    width=1000,
    height=500,
    legend_title_text="Cases",
    legend=dict(
        x=0.5,
        y=-0.2,
        xanchor="center",
        yanchor="top",
        orientation="h",
    ),
)

fig.show()


In [ ]:
# Mole fraction profiles for Case 2b
N_TIME_SAMPLES = 10

U_hist = case2b["U_hist"]
t_hist = np.asarray(case2b["t_hist"])
n_steps = len(U_hist)
sample_count = min(N_TIME_SAMPLES, n_steps)
indices = np.linspace(0, n_steps - 1, sample_count, dtype=int)
indices = np.unique(indices)

species_colors = {"N2": "#1f77b4", "O2": "#ff7f0e"}

fig = go.Figure()

for idx in indices:
    t = float(t_hist[idx])
    is_final = idx == indices[-1]
    y, X, sp_names = ff_helpers.extract_mole_fraction_profile_from_U(
        case2b, U_hist[idx]
    )
    for s_idx, sp in enumerate(sp_names):
        if sp not in ("N2", "O2"):
            continue
        color = species_colors.get(sp, "#333333")
        fig.add_trace(
            go.Scatter(
                x=X[:, s_idx],
                y=y,
                mode="lines",
                name=f"{sp}",
                legendgroup=sp,
                showlegend=bool(is_final),
                opacity=1.0 if is_final else 0.35,
                line=dict(
                    width=2 if is_final else 1,
                    color=color,
                ),
            )
        )

fig.update_xaxes(title_text="Mole fraction X_s")
fig.update_yaxes(title_text="y / H")
fig.update_layout(
    width=600,
    height=500,
    title="Mole fractions – Case 2b",
    legend=dict(x=0.5, y=-0.15, xanchor="center", yanchor="top", orientation="h"),
)
fig.show()


In [ ]:
# Thermal conductivity profiles for Case 2b
N_TIME_SAMPLES = 10

U_hist = case2b["U_hist"]
t_hist = np.asarray(case2b["t_hist"])
n_steps = len(U_hist)
sample_count = min(N_TIME_SAMPLES, n_steps)
indices = np.linspace(0, n_steps - 1, sample_count, dtype=int)
indices = np.unique(indices)

fig = make_subplots(
    rows=1,
    cols=2,
    shared_yaxes=True,
    subplot_titles=("eta_tr (trans-rot)", "eta_v (vib)"),
)

for idx in indices:
    t = float(t_hist[idx])
    is_final = idx == indices[-1]
    y, eta_tr, eta_v = ff_helpers.extract_thermal_conductivity_profile_from_U(
        case2b, U_hist[idx]
    )
    fig.add_trace(
        go.Scatter(
            x=eta_tr,
            y=y,
            mode="lines",
            name=f"t={t:.2e}s",
            legendgroup="eta_tr",
            showlegend=bool(is_final),
            opacity=1.0 if is_final else 0.35,
            line=dict(width=2 if is_final else 1, color="#1f77b4"),
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=eta_v,
            y=y,
            mode="lines",
            name=f"t={t:.2e}s",
            legendgroup="eta_v",
            showlegend=False,
            opacity=1.0 if is_final else 0.35,
            line=dict(width=2 if is_final else 1, color="#ff7f0e"),
        ),
        row=1,
        col=2,
    )

fig.update_xaxes(title_text="eta_tr [W/(m K)]", row=1, col=1)
fig.update_xaxes(title_text="eta_v [W/(m K)]", row=1, col=2)
fig.update_yaxes(title_text="y / H", row=1, col=1)
fig.update_layout(
    width=1000,
    height=500,
    title="Thermal conductivity – Case 2b",
    legend=dict(x=0.5, y=-0.15, xanchor="center", yanchor="top", orientation="h"),
)
fig.show()
